The dataset consisted of 1376 images, 690 face images with masks and 686 without masks. The original dataset is prepared by [Prajna Bhandary](https://www.linkedin.com/in/prajna-bhandary-0b03a416a/) and available at [Github](https://github.com/prajnasb/observations/tree/master/experiements/data)

Motivated by the works of :

1. [Adrian Rosebrock](https://www.pyimagesearch.com/2020/05/04/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/)
2. [sentdex](https://pythonprogramming.net/convolutional-neural-network-deep-learning-python-tensorflow-keras/)
3. [Prajna Bhandary](https://github.com/prajnasb/observations/tree/master/experiements/data)

all required libarries are imported

In [ ]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

data preprocessing : augmentation using ImageDataGenerator()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
import matplotlib as mt
train_datagen=ImageDataGenerator()

test_datagen= ImageDataGenerator(rescale=1./255)



importing the train and test data 

In [ ]:
batch_size=10
train_data=train_datagen.flow_from_directory(
'C:/Users/Aafreen/Desktop/data/train/',
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')

Found 711 images belonging to 2 classes.


In [ ]:
test_data=test_datagen.flow_from_directory(
'C:/Users/Aafreen/Desktop/data/test/',
target_size=(150,150),
batch_size=batch_size,
class_mode='binary')

Found 194 images belonging to 2 classes.


building our CNN architecture : a very simple 2 layer architecture

In [ ]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',
                             verbose=0,save_best_only=True,mode='auto')

In [ ]:
histroy=model.fit_generator(
                            train_data,steps_per_epoch=50//batch_size,epochs=10,
                             validation_data=test_data,validation_steps=40//batch_size)

Epoch 1/10
5/5 [==============================] - 2s 463ms/step - loss: 7.6685 - acc: 0.5000 - val_loss: 7.6620 - val_acc: 0.5750
Epoch 2/10
5/5 [==============================] - 2s 448ms/step - loss: 7.6703 - acc: 0.4800 - val_loss: 7.6663 - val_acc: 0.5250
Epoch 3/10
5/5 [==============================] - 2s 458ms/step - loss: 7.6703 - acc: 0.4800 - val_loss: 7.6663 - val_acc: 0.5250
Epoch 4/10
5/5 [==============================] - 2s 453ms/step - loss: 7.6685 - acc: 0.5000 - val_loss: 7.6707 - val_acc: 0.4750
Epoch 5/10
5/5 [==============================] - 2s 477ms/step - loss: 7.6826 - acc: 0.3400 - val_loss: 7.6773 - val_acc: 0.4000
Epoch 6/10
5/5 [==============================] - 2s 383ms/step - loss: 7.6739 - acc: 0.4390 - val_loss: 7.6751 - val_acc: 0.4250
Epoch 7/10
5/5 [==============================] - 2s 454ms/step - loss: 7.6721 - acc: 0.4600 - val_loss: 7.6685 - val_acc: 0.5000
Epoch 8/10
5/5 [==============================] - 2s 452ms/step - loss: 7.6738 - acc: 0.44

stored the trained model in the following keras file 

In [ ]:
model.save('mask_project.h5')

importing the OpenCV framework for testing our trained model in real time . Also making use of keras as interface for the TensorFlow library

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('mask_project.h5')

We will implement our use case using the Haar Cascade classifier. Haar Cascade classifier is an effective object detection approach which was proposed by Paul Viola and Michael Jones.

So how this works is they are huge individual .xml files with a lot of feature sets and each xml corresponds to a very specific type of use case. This particular xml file is then used extensively for detecting face

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model = tf.keras.models.load_model('mask_project.h5')

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

this project will help us detect through a video feed real time on whether a person is wearing a mask (1) or not wearing a mask (0)